# Annex: Explanation of the Vectorised version of our Gradient Descent function

Below is the gradient descent function that we have vectorised taking advantage of Numpy.

We use Numpy as this way we can take advantage of the parallel processing functions of our CPU to make calculations faster. As an extra bonus the function is defined in a more generic term, as we can then use matrices of any size and the function will respond with the right number of parameters.

In [2]:
import numpy as np

In [3]:
def GradientDescent_np(X, y, max_iterations=10000, lr=0.1, epsilon = 0.000001):
    m, n = X.shape # number of samples, number of features
    J = []

    #initialize the weights to zero (or anything else you want, e.g. a small random value)
    w = np.zeros(n)
    
    # Repeat until convergence (or max_iterations)
    for iteration in range(max_iterations):
        grad = np.dot(X.T , (np.dot(X, w) - y)) / m;
        w = w - lr*grad
        J.append(sum( (np.dot(X, w) - y)**2) / m)
        
        # Stopping Criterion
        if (iteration > 2) and ( abs(J[-2] - J[-1]) < epsilon):
            print ("Gradient Descent converged after {} iterations".format(iteration))
            break
    return w, J

Let me take some time to explain you the line where we calculate the gradients, as it might not be very obvious how we do this by multiplying matrices. I am talking about the line:

`grad = np.dot(X.T , (np.dot(X, w) - y)) / m;`

Where `X` is the input (the Design Matrix) `y` is the vector column of the output, and `w` is a vector column of the parameters.

The design matrix `X` for a dataset of of $m$ samples (data points) and $n$ features, should have $m$ rows and $n+1$ columns, and looks as follows:

\begin{equation*}
X = 
\begin{bmatrix}
x_0^{(1)} & x_1^{(1)} & \cdots & x_n^{(1)} \\
x_0^{(2)} & x_1^{(2)} & \cdots & x_n^{(2)} \\
\vdots  & \vdots  & \ddots & \vdots  \\
x_0^{(m)} & x_1^{(m)} & \cdots & x_n^{(m)} 
\end{bmatrix}
\end{equation*}

Each row corresponds to a data point, and each column corresponds to a feature.

Our outputs (labels, correct values, ground truth) `y` is a column vector of as many rows as data points:

\begin{equation*}
y = 
\begin{bmatrix}
y^{(1)} \\
y^{(2)} \\
\vdots   \\
y^{(m)} \\
\end{bmatrix}
\end{equation*}

The number of parameters would be equal to the number of features plus the bias (corresponding to the ficticious feature $x_0$, and we can write it as a vector `w` as follows:

\begin{equation*}
w = 
\begin{bmatrix}
w_0 \\
w_1 \\
\vdots   \\
w_n \\
\end{bmatrix}
\end{equation*}


What we want to calculate is the gradient (partial derivatives) for all thetas from $w_0$ to $w_n$. These will be used afterwards to change each of the thetas accordingly. These are given by 

$\frac{\partial}{\partial w_0} J(w) = {1 \over m} \sum_{i=1}^m{(f_w(x^{(i)}) - y^{(i)}) x_0^{(i)}}$

$\frac{\partial}{\partial w_1} J(w) = {1 \over m} \sum_{i=1}^m{(f_w(x^{(i)}) - y^{(i)}) x_1^{(i)}}$

$\vdots$

$\frac{\partial}{\partial w_n} J(w) = {1 \over m} \sum_{i=1}^m{(f_w(x^{(i)}) - y^{(i)}) x_n^{(i)}}$

So, what we would like at each iteration is to calculate a list of partial derivatives (gradients). This is what our vector `grad` will contain at the end of the operation. And we will do it in a single line of code, taking advantage of the linear algebra capabilities of NumPy:

`grad = np.dot(X.T , (np.dot(X, w) - y)) / m;`

I will start explaining from the inside out. First let's see what the part `np.dot(X, w)` does. This is a multiplication of the matrix `X` with the vector `w` and it gives us the $f_{w}$ for each of the points in our dataset.

\begin{equation*}
X w = 
\begin{bmatrix}
x_0^{(1)} & x_1^{(1)} & \cdots & x_n^{(1)} \\
x_0^{(2)} & x_1^{(2)} & \cdots & x_n^{(2)} \\
\vdots  & \vdots  & \ddots & \vdots  \\
x_0^{(m)} & x_1^{(m)} & \cdots & x_n^{(m)} 
\end{bmatrix}
\begin{bmatrix}
w_0 \\
w_1 \\
\vdots   \\
w_n \\
\end{bmatrix}
=
\begin{bmatrix}
f_{w}^{(1)} \\
f_{w}^{(2)} \\
\vdots   \\
f_{w}^{(m)} \\
\end{bmatrix}
\end{equation*}

Therefore the part of `np.dot(X, w) - y` is basically the residues, the differences between what we calculate and what the right value should be FOR EACH of the points of our dataset:

\begin{equation*}
Residues = 
\begin{bmatrix}
f_{w}^{(1)} \\
f_{w}^{(2)} \\
\vdots   \\
f_{w}^{(m)} \\
\end{bmatrix}
-
\begin{bmatrix}
y^{(1)} \\
y^{(2)} \\
\vdots   \\
y^{(m)} \\
\end{bmatrix}
=
\begin{bmatrix}
f_{w}^{(1)} - y^{(1)}\\
f_{w}^{(2)} - y^{(2)} \\
\vdots   \\
f_{w}^{(m)} - y^{(m)} \\
\end{bmatrix}
\end{equation*}

Now, to calculate the gradient corresponding to a specific parameter, we need to multiply each of these residues with the corresponding feature, and then sum all these multipied residues over all the points of our dataset. This is what the other dot product does: `np.dot(X.T , (np.dot(X, w) - y))`, it transposes the Design Matrix and then multiplies it with our vector of residues. Let's see what this does:

\begin{equation*}
X^T Residues = 
\begin{bmatrix}
x_0^{(1)} & x_0^{(2)} & \cdots & x_0^{(m)} \\
x_1^{(1)} & x_1^{(2)} & \cdots & x_1^{(m)} \\
\vdots  & \vdots  & \ddots & \vdots  \\
x_n^{(1)} & x_n^{(2)} & \cdots & x_n^{(m)}
\end{bmatrix}
\begin{bmatrix}
f_{w}^{(1)} - y^{(1)}\\
f_{w}^{(2)} - y^{(2)} \\
\vdots   \\
f_{w}^{(m)} - y^{(m)} \\
\end{bmatrix}
=
\begin{bmatrix}
(f_{w}^{(1)} - y^{(1)}) x_0^{(1)} + (f_{w}^{(2)} - y^{(2)}) x_0^{(2)} + \cdots + (f_{w}^{(m)} - y^{(m)}) x_0^{(m)} \\
(f_{w}^{(1)} - y^{(1)}) x_1^{(1)} + (f_{w}^{(2)} - y^{(2)}) x_1^{(2)} + \cdots + (f_{w}^{(m)} - y^{(m)}) x_1^{(m)} \\
\vdots \\
(f_{w}^{(1)} - y^{(1)}) x_1^{(m)} + (f_{w}^{(2)} - y^{(2)}) x_n^{(2)} + \cdots + (f_{w}^{(m)} - y^{(m)}) x_n^{(m)}
\end{bmatrix}
=
\begin{bmatrix}
\sum_{i=1}^m{(f_w(x^{(i)}) - y^{(i)}) x_0^{(i)}} \\
\sum_{i=1}^m{(f_w(x^{(i)}) - y^{(i)}) x_1^{(i)}} \\
\vdots  \\
\sum_{i=1}^m{(f_w(x^{(i)}) - y^{(i)}) x_n^{(i)}} \\
\end{bmatrix}
\end{equation*}

The only thing remaining is to divide everything with m, so the final result of this line is a column vector with the following contents:

`grad = np.dot(X.T , (np.dot(X, w) - y)) / m;`

\begin{equation*}
grad = 
\begin{bmatrix}
{1 \over m} \sum_{i=1}^m{(f_w(x^{(i)}) - y^{(i)}) x_0^{(i)}} \\
{1 \over m} \sum_{i=1}^m{(f_w(x^{(i)}) - y^{(i)}) x_1^{(i)}} \\
\vdots  \\
{1 \over m} \sum_{i=1}^m{(f_w(x^{(i)}) - y^{(i)}) x_n^{(i)}} \\
\end{bmatrix}
\end{equation*}